## Generate Employee Account

In [1]:
import pandas as pd
from faker import Faker
import random
import uuid
import psycopg2
from psycopg2.extras import execute_values

fake = Faker("vi_VN")

In [3]:
employee_account = []
account_id = []


used_emails = set()
used_usernames = set()

while len(employee_account) < 300:
    username = fake.user_name()
    email = fake.email()

    if username in used_usernames or email in used_emails:
        continue

    used_usernames.add(username)
    used_emails.add(email)
    
    account = {
        "username": fake.user_name(),
        "password": "User@123",
        "email": fake.email(),
        "role_id": "909ed248-5a65-4d9e-aac9-da41c83e38a4"
    }
    employee_account.append(account)

print(len(employee_account))

300


In [4]:
conn = psycopg2.connect(
    host="aws-1-ap-south-1.pooler.supabase.com",
    database="postgres",
    user="postgres.rruavcjmtgpxyznzwkhw",
    password="khaibaolocnguyen",
    port=5432
)
cur = conn.cursor()

query = """
INSERT INTO account (username, password, email, role_id)
VALUES %s
RETURNING account_id
"""

values = [
    (a["username"], a["password"], a["email"], a["role_id"])
    for a in employee_account
]

execute_values(cur, query, values, page_size=len(values))

account_id = [row[0] for row in cur.fetchall()]

conn.commit()

In [5]:
print(len(employee_account))
print(len(account_id))

300
300


## Generate Employee 

In [6]:
employee = []
employee_id = []

gender_options = [
    "male", "female"
]

for i in range(300):
    emp = {
        "employee_name": fake.name(),
        "date_of_birth": fake.date_of_birth(minimum_age=18, maximum_age=60),
        "gender": random.choice(gender_options),
        "manager_id": "7ce3bbbe-c72d-442b-bcf3-8d797331e95d",
        "account_id": account_id[i]
    }
    employee.append(emp)

In [7]:
print(len(employee))

300


In [8]:
query = """
INSERT INTO employee (employee_name, date_of_birth, gender, manager_id, account_id)
VALUES %s
RETURNING employee_id
"""

values = [
    (a["employee_name"], a["date_of_birth"], a["gender"], a["manager_id"], a["account_id"])
    for a in employee
]

execute_values(cur, query, values, page_size=len(values))

employee_id = [row[0] for row in cur.fetchall()]

conn.commit()

## Assign these employee to be a veterinarian

In [9]:
print(len(employee_id))

300


In [10]:
veterinarian = []
for i in range(300):
    vet = {
        "employee_id": employee_id[i],
        "degree": random.choice([
          "Bachelor of Veterinary Medicine",
          "Doctor of Veterinary Medicine",
          "Master of Veterinary Science"
        ]),
        "specialization": random.choice([
          "Surgery", "Dermatology", "Dentistry", "Internal Medicine", "Radiology",
          "Pediatrics", "Obstetrics and Gynecology", "Cardiology", "Neurology", 
          "Ophthalmology", "Psychiatry", "Orthopedics", "Oncology", "Endocrinology", 
          "Gastroenterology", "Hematology", "Otolaryngology", "Urology", 
          "Anesthesiology", "Emergency Medicine"
        ])
    }
    veterinarian.append(vet)

In [11]:
query = """
INSERT INTO veterinarian (employee_id, degree, specialization)
VALUES %s
"""

values = [
    (a["employee_id"], a["degree"], a["specialization"])
    for a in veterinarian
]

execute_values(cur, query, values, page_size=len(values))

conn.commit()